In [1]:
import pandas as pd
import numpy as np

# Initialization

Create the base network as a matrix of costs, where a cost of NaN represents a non-adjacent node and a cost of 0 represents cost to the same node

In [2]:
network = pd.DataFrame(
    [[np.nan,5,1,np.nan,np.nan,np.nan],
     [np.nan,np.nan,4,6,np.nan,np.nan],
     [np.nan,np.nan,np.nan,7,2,np.nan],
     [np.nan,np.nan,np.nan,np.nan,np.nan,2],
     [np.nan,1,np.nan,3,np.nan,8],
     [np.nan,np.nan,np.nan,np.nan,np.nan,np.nan]])

In [3]:
display(network)

,0,1,2,3,4,5
0,NaN,5.0,1.0,NaN,NaN,NaN
1,NaN,NaN,4.0,6.0,NaN,NaN
2,NaN,NaN,NaN,7.0,2.0,NaN
3,NaN,NaN,NaN,NaN,NaN,2.0
4,NaN,1.0,NaN,3.0,NaN,8.0
5,NaN,NaN,NaN,NaN,NaN,NaN


## Djikstra's Algorithm

In [4]:
# Initialize the temporary and permanent labels for optimal costs
permanent = pd.Series([0,np.nan,np.nan,np.nan,np.nan,np.nan])
temp = pd.Series([np.nan,np.nan,np.nan,np.nan,np.nan,np.nan])

In [ ]:
previousPermanentNodeIndex = 0

while permanent.isnull().values.any():
    # calculate the new possible node costs using the previous permanent node
    newNodeCosts = network.iloc[previousPermanentNodeIndex] + permanent[previousPermanentNodeIndex]

    # find min between new possible node costs and temporary costs, call it calcCosts
    calcCosts = np.fmin(newNodeCosts, temp)

    # find min of the calcCosts and the associated node, make the node
    # and the cost permanent
    newMin = np.nanmin(calcCosts)
    newMinIndex = np.argmin(calcCosts)

    # update the permanent costs vector
    permanent[newMinIndex] = newMin
    print(permanent)

    # update the temporary costs vector
    temp = np.fmin(calcCosts, temp)
    # make sure that any permanent nodes are't included
    temp[permanent.notna()] = np.nan

    # update the previousPermanentNodeIndex
    previousPermanentNodeIndex = newMinIndex

0    0.0
1    NaN
2    1.0
3    NaN
4    NaN
5    NaN
dtype: float64
0    0.0
1    NaN
2    1.0
3    NaN
4    3.0
5    NaN
dtype: float64
0    0.0
1    4.0
2    1.0
3    NaN
4    3.0
5    NaN
dtype: float64
0    0.0
1    4.0
2    1.0
3    6.0
4    3.0
5    NaN
dtype: float64
0    0.0
1    4.0
2    1.0
3    6.0
4    3.0
5    8.0
dtype: float64


## Bellman-Ford Algorithm